<br>

**Herein**, a varying intercept & varying gradient stochastic regression model wherein the dependent variables are modelled as covarying series.

<br>

# Preliminaries

Cleaning-up

In [ ]:
!rm -rf *.sh

<br>

**Packages**

In [ ]:
import subprocess

In [ ]:
if 'google.colab' in str(get_ipython()):
    subprocess.run('wget -q https://raw.githubusercontent.com/plausibilities/sars/develop/scripts.sh', shell=True)
    subprocess.run('chmod u+x scripts.sh', shell=True)
    subprocess.run('./scripts.sh', shell=True)

<br>

**Paths**

In [ ]:
import os
import pathlib
import sys

In [ ]:
if not 'google.colab' in str(get_ipython()):
    
    notebooks = os.path.split(os.getcwd())[0]
    parent = str(pathlib.Path(notebooks).parent)
    sys.path.append(parent)


In [ ]:
warehouse = 'natural'

if not os.path.exists(warehouse):
    os.makedirs(warehouse)

<br>

**Libraries**

In [ ]:
%matplotlib inline

In [ ]:
import arviz as az
import graphviz
import pymc3 as pm

import logging
import collections

import numpy as np
import pandas as pd
import scipy as sp
import math

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import theano


<br>

Custom

In [ ]:
import sars.graphics.settings
import sars.graphics.relational

<br>

**Generic Instances**

In [ ]:
relational = sars.graphics.relational.Relational()

RelationalGraphLabels = collections.namedtuple(typename='RelationalGraphLabels',
                                               field_names=['title', 'xlabel', 'ylabel'])

<br>

## Set-up

**Display**<br>
If the display screen supports **retina display**, a figure's resolution can be enhanced via


In [ ]:
%config InlineBackend.figure_format = 'retina'

<br>

**Jupyter Settings**

In [ ]:
settings = sars.graphics.settings.Settings()

settings.layout()
settings.aesthetics()

<br>

## Logging

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>
<br>

# Data

## Baseline

In [ ]:
import sars.models.inpatients.src.baseline

In [ ]:
baseline = sars.models.inpatients.src.baseline.Baseline().exc()

In [ ]:
baseline.rename(columns={"datetimeobject": "date"}, inplace=True)

In [ ]:
logger.info('\n{}\n'.format(baseline.tail()))

<br>
<br>

## Focus

In [ ]:
Futures = collections.namedtuple(typename='Futures', field_names=['steps', 'ahead'])
futures = Futures._make([np.array([1]), 21])

In [ ]:
Variables = collections.namedtuple(typename='Variables', field_names=['independent', 'dependent', 'label', 'independent_'])
variables = Variables._make((['ndays'], ['positiveCumulative', 'hospitalizedCumulative', 'deathCumulative'], 
                             ['positives [C]', 'hospitalized [C]', 'deaths [C]'], 'date'))

<br>

**State**

In [ ]:
stusps = 'ND'

<br>

**Data**

* N: The number of records
* M: The number of independent variables
* P: The number of dependent variables

In [ ]:
import sars.models.inpatients.src.focus

In [ ]:
data, parameters = sars.models.inpatients.src.focus.Focus(
    baseline=baseline, variables=variables, futures=futures).exc(stusps=stusps)

logger.info('The fields of data\n{}\n'.format(data._fields))
logger.info('The fields of parameters\n{}\n'.format(parameters._fields))

<br>

**Illustration**

In [ ]:
ax = relational.figure(width=4.7, height=3.5)
ax.set_prop_cycle(color=['black', 'blue', 'red'])

ax.plot(data.independent_, np.log(data.dependent), '-')

ax.tick_params(axis='x', labelrotation=90)
relational.annotation(handle=ax, 
                      labels=RelationalGraphLabels._make(['\ndata\n', '\ndate', 'ln(observations [C])\n']))

ax.legend(labels=variables.label, loc='upper left', fontsize='small');                      

<br>
<br>

# Inference

## Scaled Tensors

In [ ]:
import sars.models.inpatients.scaled

In [ ]:
scaled = sars.models.inpatients.scaled.Scaled(data=data).exc()

logger.info('scaled\n{}\n'.format(scaled._fields))

<br>

## Elements

In [ ]:
import sars.models.inpatients.elements

In [ ]:
sections_ = scaled.independent.shape[0]
instances_ = np.ones(shape=scaled.independent.shape[0], dtype=int)

elements = sars.models.inpatients.elements.Elements(sections_=sections_, instances_=instances_).exc()

logger.info('elements\n{}\n'.format(elements._fields))

<br>
<br>

## Model

<br>

**Inference**

In [ ]:
import sars.models.inpatients.natural.inference

In [ ]:
inference = sars.models.inpatients.natural.inference.Inference(data=scaled, parameters=parameters, elements=elements)

<br>

Inference Model Characteristics

In [ ]:
imc = inference.exc()

In [ ]:
imc._fields

<br>

**DAG**

* http://www.graphviz.org/doc/info/attrs.html

In [ ]:
# The DAG
diagram = pm.model_graph.ModelGraph(model=imc.model).make_graph()
diagram.node_attr.update(shape='circle')
diagram.graph_attr.update(size="5.7,7.9")

# Diagrams
diagram.save(os.path.join(warehouse, 'model.gv'))
graphviz.render(engine='dot', format='pdf', filepath=os.path.join(warehouse, 'model.gv'))

graphviz.Source.from_file(filename=os.path.join(warehouse, 'model.gv'))

<br>

**Trace**

In [ ]:
imc.trace.varnames

<br>

Samplings

In [ ]:
samplings = imc.trace.report.n_draws * imc.trace.nchains
logger.info('The number of sampled instances\n{}\n'.format(samplings))

<br>
<br>

# Analysis

## Estimates

In [ ]:
import sars.models.inpatients.estimates

In [ ]:
estimates = sars.models.inpatients.estimates.Estimates(
    trace=imc.trace, futures=futures, samplings=samplings, parameters=parameters)

<br>

**Intercepts & Gradients**

In [ ]:
Arguments = collections.namedtuple(typename='Arguments', field_names=['name', 'cholesky'])

intercepts = estimates.exc(arguments=Arguments._make(('intercept', 'packed_l_c')))
gradients = estimates.exc(arguments=Arguments._make(('gradient', 'packed_l_m')))

logger.info('\n{}\n'.format(intercepts.shape))
logger.info('\n{}\n'.format(gradients.shape))

In [ ]:
InterceptsAndGradients = collections.namedtuple(typename='InterceptsAndGradients', field_names=['intercepts', 'gradients'])
iag = InterceptsAndGradients._make([intercepts, gradients])

<br>
<br>

## Predictions

In [ ]:
import sars.models.inpatients.predictions

In [ ]:
predictions = sars.models.inpatients.predictions.Predictions(
    iag=iag, samplings=samplings, scaled=scaled, parameters=parameters).exc(size=100)

In [ ]:
logger.info('\n{}\n'.format(predictions._fields))

In [ ]:
logger.info('\n{}\n'.format(predictions.line.shape))
logger.info('\n{}\n'.format(predictions.lines.shape))

<br>
<br>

# Illustrate

In [ ]:
import sars.models.inpatients.graphs

<br>

$\color{#939393}{\text{The fields}}$

Opting for **date variables**, i.e., `data.independent_ & data.abscissae_`, instead of **days thus far variables**, i.e., `data.independent & data.abscissae`, respectively.

In [ ]:
Fields = collections.namedtuple(typename='Futures', field_names=['initial', 'extended'])
fields = Fields._make((data.independent_, data.abscissae_))

<br>

$\color{#939393}{\text{The labels}}$

In [ ]:
variables.label

<br>

$\color{#939393}{\text{The graphing instance}}$

In [ ]:
graphs = sars.models.inpatients.graphs.Graphs(data=data, predictions=predictions, titles=variables.label, fields=fields)

<br>

**Together**

In [ ]:
graphs.together(ylabel='ln(observations [C])', xlabel='date')

<br>

**Separate**

In [ ]:
matplotlib.rcParams['figure.autolayout'] = False
matplotlib.rcParams['figure.constrained_layout.use'] = False
matplotlib.rcParams['figure.figsize'] = 7.1, 5.3

matplotlib.rcParams['savefig.bbox'] = 'standard'
matplotlib.rcParams["savefig.pad_inches"] = 0.5

In [ ]:
fig, handle = graphs.separate(adjust=np.array([5, 5]), layout=np.array([5, 7]), xlabel='date')

In [ ]:
fig.savefig(os.path.join(warehouse, 'natural.png'), transparent=False, 
            pad_inches=0.25,
            bbox_inches=matplotlib.transforms.Bbox([[0, 0], [9.0, 5.5]]).expanded(sw=1.2, sh=1.0))